# Setup

In [ ]:
!pip install betterreads
from betterreads import client

  Created wheel for rauth: filename=rauth-0.7.3-cp36-none-any.whl size=16055 sha256=c3767ca8730ec5618db93ee8908c432065a8540cfd4e60e533ab2311704e2549
  Stored in directory: /root/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
  Created wheel for backports-datetime-fromisoformat: filename=backports_datetime_fromisoformat-1.0.0-cp36-cp36m-linux_x86_64.whl size=32337 sha256=8ec77f2e46b847c9be3438a227f0b5c324224c0ad9376bc9d473195f7ae0cc67
  Stored in directory: /root/.cache/pip/wheels/45/ca/6c/8b422f60f0660a8e23f081581fabbbbde5c4d0e2683f8050c0
Successfully built rauth backports-datetime-fromisoformat


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import bs4
import requests
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import itertools
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Getting Data using link scraper and GoodReads API 

In [ ]:
# Setting up the Goodreads API
KEY='eBp9c4mms5adGETXyWcnaA'
SECRET='fet3Al3l2YzWuUILNe0pQt4vaJijz0szhBi0GcSP8'
gc = client.GoodreadsClient(KEY, SECRET)

## Best books of the 21st Century

In [ ]:
# Getting all the links on list for the best 5200 books of the 21st century
profile_list=[]
# Going through each page (1-52) of the best books of the 21st century 
for page in range(1,53):
  url = 'https://www.goodreads.com/list/show/7.Best_Books_of_the_21st_Century?page='+str(page)
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.text, 'html.parser') 
  links = soup.find_all('a')
  # Gets all the links for the books
  for link in links:
    if ('/book/show/' in link.get('href',[])):
      profilelink = link.get('href') 
      if profilelink not in profile_list and 'https://www.goodreads.com/' not in profilelink:
        profile_list.append(profilelink)
#Going through the links to get the bookid
book_list=[]
for i in range(len(profile_list)):  
  split_link=profile_list[i].split('/')
  bookid=split_link[3].split('-')
  bookid=bookid[0].split('.')[0]
  book_list.append(bookid)

In [ ]:
# Using the Goodreads API to get information about the books using the bookid
# Setting up the dataframe for the Goodreads book information
gr_df = pd.DataFrame(columns = ['gr_id','title' ,'author' , 'publisher', 'isbn', 'language', 'description' ,
                                'avg_rating','ratings_count', 'review_count' , 
                                'url'])
# Going through all the books and getting the information for each book using the api
# Placing it into the dataframe
for id in book_list:
  book = gc.book(id)
  book_id = book.gid
  title = book.title
  author = book.authors[0]
  publisher = book.publisher
  isbn = book.isbn
  lang = book.language_code
  description = book.description
  avg_rating = book.average_rating
  rating_count = book.ratings_count
  review_count = book.text_reviews_count
  url = book.link
  gr_df.loc[len(gr_df)] = [book_id, title, author, publisher, isbn, lang, description, avg_rating, rating_count, review_count, url]

In [ ]:
# Saving the dataframe as a csv file
gr_df.to_csv("gr_df.csv")

## Best YA Books

In [ ]:
# Getting all the links on list for the best young adult books (5200)
profile_list=[]
# Going through each page (1-52) of the best young adult books
for page in range(1,53):
  url = 'https://www.goodreads.com/list/show/43.Best_Young_Adult_Books?page='+str(page)
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.text, 'html.parser') 
  links = soup.find_all('a')
  # Gets all the links for the books
  for link in links:
    if ('/book/show/' in link.get('href',[])):
      profilelink = link.get('href') 
      if profilelink not in profile_list and 'https://www.goodreads.com/' not in profilelink:
        profile_list.append(profilelink)
#Going through the links to get the bookid
book_list=[]
for i in range(len(profile_list)):  
  split_link=profile_list[i].split('/')
  bookid=split_link[3].split('-')
  bookid=bookid[0].split('.')[0]
  book_list.append(bookid)

In [ ]:
# Using the Goodreads API to get information about the books using the bookid
# Setting up the dataframe for the Goodreads book information
gr_df_ya = pd.DataFrame(columns = ['gr_id','title' ,'author' , 'publisher', 'isbn', 'language', 'description' ,
                                'avg_rating','ratings_count', 'review_count' , 
                                'url'])
# Going through all the books and getting the information for each book using the api
# Placing it into the dataframe
for id in book_list:
  book = gc.book(id)
  book_id = book.gid
  title = book.title
  author = book.authors[0]
  publisher = book.publisher
  isbn = book.isbn
  lang = book.language_code
  description = book.description
  avg_rating = book.average_rating
  rating_count = book.ratings_count
  review_count = book.text_reviews_count
  url = book.link
  gr_df_ya.loc[len(gr_df_ya)] = [book_id, title, author, publisher, isbn, lang, description, avg_rating, rating_count, review_count, url]

In [ ]:
# Saving the dataframe as a csv file
gr_df_ya.to_csv("gr_df_ya.csv")

## Best Sci-Fi Books (DID NOT WORK)

In [ ]:
# Getting all the links on list for the Best Science Fiction/Fanstasy books (5200)
profile_list=[]
# Going through each page (1-52) of the Best Science Fiction/Fanstasy books
for page in range(1,53):
  url = 'https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books?page='+str(page)
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.text, 'html.parser') 
  links = soup.find_all('a')
  # Gets all the links for the books
  for link in links:
    if ('/book/show/' in link.get('href',[])):
      profilelink = link.get('href') 
      if profilelink not in profile_list and 'https://www.goodreads.com/' not in profilelink:
        profile_list.append(profilelink)
#Going through the links to get the bookid
book_list=[]
for i in range(len(profile_list)):  
  split_link=profile_list[i].split('/')
  bookid=split_link[3].split('-')
  bookid=bookid[0].split('.')[0]
  book_list.append(bookid)

In [ ]:
# Using the Goodreads API to get information about the books using the bookid
# Setting up the dataframe for the Goodreads book information
gr_df_sf = pd.DataFrame(columns = ['gr_id','title' ,'author' , 'publisher', 'isbn', 'language', 'description' ,
                                'avg_rating','ratings_count', 'review_count' , 
                                'url'])
# Going through all the books and getting the information for each book using the api
# Placing it into the dataframe
for id in book_list:
  book = gc.book(id)
  book_id = book.gid
  title = book.title
  author = book.authors[0]
  publisher = book.publisher
  isbn = book.isbn
  lang = book.language_code
  description = book.description
  avg_rating = book.average_rating
  rating_count = book.ratings_count
  review_count = book.text_reviews_count
  url = book.link
  gr_df_sf.loc[len(gr_df_sf)] = [book_id, title, author, publisher, isbn, lang, description, avg_rating, rating_count, review_count, url]

In [ ]:
# Saving the dataframe as a csv file
gr_df_sf.to_csv("gr_df_sf.csv")

## Best Mystery Books

In [ ]:
# Getting all the links on list for the best mystery books (5200)
profile_list=[]
# Going through each page (1-52) of the best mystery books
for page in range(1,53):
  url = 'https://www.goodreads.com/list/show/11.Best_Crime_Mystery_Books?page='+str(page)
  response = requests.get(url)
  soup = bs4.BeautifulSoup(response.text, 'html.parser') 
  links = soup.find_all('a')
  # Gets all the links for the books
  for link in links:
    if ('/book/show/' in link.get('href',[])):
      profilelink = link.get('href') 
      if profilelink not in profile_list and 'https://www.goodreads.com/' not in profilelink:
        profile_list.append(profilelink)
#Going through the links to get the bookid
book_list=[]
for i in range(len(profile_list)):  
  split_link=profile_list[i].split('/')
  bookid=split_link[3].split('-')
  bookid=bookid[0].split('.')[0]
  book_list.append(bookid)

In [ ]:
# Using the Goodreads API to get information about the books using the bookid
# Setting up the dataframe for the Goodreads book information
gr_df_mystery = pd.DataFrame(columns = ['gr_id','title' ,'author' , 'publisher', 'isbn', 'language', 'description' ,
                                'avg_rating','ratings_count', 'review_count' , 
                                'url'])
# Going through all the books and getting the information for each book using the api
# Placing it into the dataframe
for id in book_list:
  book = gc.book(id)
  book_id = book.gid
  title = book.title
  author = book.authors[0]
  publisher = book.publisher
  isbn = book.isbn
  lang = book.language_code
  description = book.description
  avg_rating = book.average_rating
  rating_count = book.ratings_count
  review_count = book.text_reviews_count
  url = book.link
  gr_df_mystery.loc[len(gr_df_mystery)] = [book_id, title, author, publisher, isbn, lang, description, avg_rating, rating_count, review_count, url]

In [ ]:
# Saving the dataframe as a csv file
gr_df_mystery.to_csv("gr_df_mystery.csv")

# Cleaning Data

## Uploading CSV file

In [ ]:
# Uploading the dataframe from the csv file
ya_df = pd.read_csv('gr_df_ya.csv', index_col=0)
sci_fi_df = pd.read_csv('gr_df_sf.csv', index_col=0)
mystery_df = pd.read_csv('gr_df_mystery.csv', index_col=0)

## Filtering Dataframe Data

In [ ]:
# Dropping any rows of books that does not include a publisher or description
ya_df.dropna(subset=['publisher', 'description'],inplace=True)
sci_fi_df.dropna(subset=['publisher', 'description'],inplace=True)
mystery_df.dropna(subset=['publisher', 'description'],inplace=True)

# Recasting the Description information as a string for text processing
ya_df['description']=ya_df['description'].astype(str)
sci_fi_df['description']=sci_fi_df['description'].astype(str)
mystery_df['description']=mystery_df['description'].astype(str)

# Dropping all other types of books other than english to look at get common english topics
ya_df.drop(ya_df.index[ya_df['language'] != 'eng'], inplace = True)
sci_fi_df.drop(sci_fi_df.index[sci_fi_df['language'] != 'eng'], inplace = True)
mystery_df.drop(mystery_df.index[mystery_df['language'] != 'eng'], inplace = True)


## Cleaning Description Data

In [ ]:
stop_words = stopwords.words("english")
wordnet = WordNetLemmatizer()

def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = bs4.BeautifulSoup(x, "lxml").text
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [ ]:
ya_df['clean_text'] = ya_df.description.apply(text_preproc)
sci_fi_df['clean_text'] = sci_fi_df.description.apply(text_preproc)
mystery_df['clean_text'] = mystery_df.description.apply(text_preproc)

In [ ]:
# Saving the dataframe as a csv file
ya_df.to_csv("ya_df_clean.csv")
sci_fi_df.to_csv("sci_fi_df_clean.csv")
mystery_df.to_csv("mystery_df_clean.csv")

# Data Exploration
Looking at the information for the dataset

Selected Books include:
* English
* No null values in the Publisher and Description Section

In [ ]:
print("Number of Best Books of 21st Century: "+ str(len(gr_df)))
print("Number of Best Young Adult Books: "+ str(len(ya_df)))
print("Number of Best Sci-Fi Books: "+ str(len(sci_fi_df)))
print("Number of Best Mystery Books: "+ str(len(mystery_df)))

Number of Best Books of 21st Century: 3363
Number of Best Young Adult Books: 3441
Number of Best Sci-Fi Books: 3096
Number of Best Mystery Books: 3099


In [ ]:
print("Number of Different Publishers in Best Books of 21st Century: "+ str(len(gr_df['publisher'].unique())))
print("Number of Different Publishers in Best Young Adult Books: "+ str(len(ya_df['publisher'].unique())))
print("Number of Different Publishers in Best Sci-Fi Books: "+ str(len(sci_fi_df['publisher'].unique())))
print("Number of Different Publishers in Best Mystery Books: "+ str(len(mystery_df['publisher'].unique())))

Number of Different Publishers in Best Books of 21st Century: 1182
Number of Different Publishers in Best Young Adult Books: 1321
Number of Different Publishers in Best Sci-Fi Books: 1210
Number of Different Publishers in Best Mystery Books: 1032


In [ ]:
print("Number of Different Authors in Best Books of 21st Century: "+ str(len(gr_df['author'].unique())))
print("Number of Different Authors in Best Young Adult Books: "+ str(len(ya_df['author'].unique())))
print("Number of Different Authors in Best Sci-Fi Books: "+ str(len(sci_fi_df['author'].unique())))
print("Number of Different Authors in Best Mystery Books: "+ str(len(mystery_df['author'].unique())))

Number of Different Authors in Best Books of 21st Century: 2028
Number of Different Authors in Best Young Adult Books: 2301
Number of Different Authors in Best Sci-Fi Books: 1592
Number of Different Authors in Best Mystery Books: 1577


In [ ]:
# The Number of titles per publisher for Best Books of 21st Century
gr_df.groupby(by=["publisher"])['title'].count().sort_values(ascending=False)

publisher
Random House                          48
HarperCollins                         43
Knopf                                 39
Little, Brown and Company             39
Harper                                38
                                      ..
Michelle Diener                        1
Michael Joseph LTD                     1
Metropolitan Books/Henry Holt (NY)     1
Melville House Publishing              1
10Publishing                           1
Name: title, Length: 1182, dtype: int64

In [ ]:
# The Number of titles per publisher for Young Adult Books
ya_df.groupby(by=["publisher"])['title'].count().sort_values(ascending=False)

publisher
HarperTeen                               74
HarperCollins                            70
Simon Pulse                              69
Little, Brown Books for Young Readers    64
Puffin Books                             41
                                         ..
Not a Pipe Publishing                     1
Not Avail                                 1
Nostr Publishing                          1
Norilana Books                            1
108 Classic Books                         1
Name: title, Length: 1321, dtype: int64

In [ ]:
# The Number of titles per publisher for Sci-Fi Books
sci_fi_df.groupby(by=["publisher"])['title'].count().sort_values(ascending=False)

publisher
Del Rey                                    98
Tor Books                                  91
Ace                                        76
Orbit                                      58
Gollancz                                   42
                                           ..
Nesfa                                       1
Nelson Doubleday/SFBC                       1
Nelson Doubleday Inc. (Garden City, NY)     1
Nelson Doubleday & Co.                      1
01Publishing                                1
Name: title, Length: 1210, dtype: int64

In [ ]:
# The Number of titles per publisher for Mystery Books 
mystery_df.groupby(by=["publisher"])['title'].count().sort_values(ascending=False)

publisher
Minotaur Books                65
Bantam                        62
Grand Central Publishing      60
Ballantine Books              53
Vintage Crime/Black Lizard    48
                              ..
Neverland Publishing           1
NeoPoiesis Press               1
Neena H. Brar                  1
Naxos Audiobooks               1
13th Sign Publishing           1
Name: title, Length: 1032, dtype: int64

# Topic Modeling

## Young Adult Books

In [ ]:
num_topics = 10

count_vectorizer = CountVectorizer(stop_words='english')
lda_data = count_vectorizer.fit_transform(ya_df.clean_text)
# Create and fit the LDA model
lda = LDA(n_components=num_topics, n_jobs=-1)
lda.fit(lda_data)
#lda.components_[0]

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, 25)

Topics found via LDA:

Topic #0:
new year school old adventure world life magic time dark years friends series evil girl alex family day love people best help lives like little

Topic #1:
life new story old time year world young love best way girl father make school family knows dark book heart people mother power friend little

Topic #2:
life world new year old book love family story way like years novel finds death time dark secret mysterious boy girl father discovers school man

Topic #3:
life world love new time young years like book story secret family way school power old friends people true know secrets future home boy girl

Topic #4:
world life new love young school year book day story way old finds time boy things family city long war dark man friends people novel

Topic #5:
world life family love old new school story year novel high best like book time little years young home finds jack girl magic make town

Topic #6:
life love new world like family school father story friend

In [ ]:
#0 - Adventure , 1 -Love , 2 - Mysterious , 3 - life , 4 - Highschool ,5 - Transition, 6 - Tragedy , 7 -Friendship , 8 - Family, 9 - Drama

In [ ]:
#Output1 - A file showing which words load on which topics, 
words = count_vectorizer.get_feature_names()
topic0 = lda.components_[0]
topic1 = lda.components_[1]
topic2 = lda.components_[2]
topic3 = lda.components_[3]
topic4 = lda.components_[4]
topic5 = lda.components_[5]
topic6 = lda.components_[6]
topic7 = lda.components_[7]
topic8 = lda.components_[8]
topic9 = lda.components_[9]
d = {'words': words,
     'Adventure': topic0,
     'Love':topic1,
     'Mysterious': topic2,
     'Life': topic3,
     'Highschool': topic4,
     'Transition': topic5,
     'Tragedy': topic6,
     'Friendship': topic7,
     'Family': topic8,
     'Drama': topic9     }
output1 = pd.DataFrame(d)
output1.head(10)

,words,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
0,aaaanah,0.100000,0.100000,1.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.1,0.100000
1,aan,0.100000,0.100000,0.100000,0.100000,0.100000,1.100000,0.100000,0.1,0.1,0.100000
2,aang,0.100041,0.100000,0.100000,0.100000,0.100000,1.099959,0.100000,0.1,0.1,0.100000
3,aarabassa,0.100000,0.100000,0.100000,0.100000,1.100000,0.100000,0.100000,0.1,0.1,0.100000
4,aaron,2.100000,1.099999,4.100003,0.100000,0.100000,0.100000,1.099998,0.1,0.1,0.100000
5,aarons,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,1.100000,0.1,0.1,0.100000
6,aarti,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,1.100000,0.1,0.1,0.100000
7,aaryn,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,2.100000,0.1,0.1,0.100000
8,aba,0.100000,0.100000,0.100000,0.100000,2.100000,0.100000,0.100000,0.1,0.1,0.100000
9,abandon,0.100000,3.099957,0.100040,0.100011,2.100002,0.100000,0.100002,0.1,0.1,4.099988


In [ ]:
weights=lda.transform(lda_data)
topic_weights=pd.DataFrame(weights).rename(columns={0:'Adventure', 1: 'Love', 2: 'Mysterious', 3:'Life', 4: 'Highschool', 5:'Transition', 6:'Tragedy', 7: 'Friendship', 8: 'Family', 9:'Drama'})
topic_weights.head(10)

,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
0,0.000918,0.000918,0.000918,0.991741,0.000918,0.000918,0.000918,0.000918,0.000918,0.000918
1,0.001299,0.988309,0.001299,0.001299,0.001299,0.001299,0.001299,0.001299,0.001299,0.001299
2,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.962493,0.004168,0.004167,0.004167
3,0.001316,0.001316,0.001316,0.001316,0.001316,0.001316,0.988155,0.001316,0.001316,0.001316
4,0.002000,0.002000,0.002001,0.002000,0.002000,0.002001,0.002001,0.981995,0.002000,0.002001
5,0.001923,0.001924,0.544024,0.001923,0.001923,0.001924,0.440588,0.001924,0.001923,0.001924
6,0.001250,0.001250,0.001250,0.001250,0.001250,0.001250,0.955859,0.034139,0.001250,0.001250
7,0.000971,0.000971,0.000971,0.000971,0.991260,0.000971,0.000971,0.000971,0.000971,0.000971
8,0.001409,0.001409,0.443141,0.001409,0.001409,0.001409,0.001409,0.001409,0.001409,0.545589
9,0.001150,0.001150,0.001150,0.001150,0.001150,0.989652,0.001150,0.001150,0.001150,0.001150


In [ ]:
threshold_topics = topic_weights.apply(lambda x: np.where(x >= 0.4, x, 0))
threshold_topics.head(10)

,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
0,0.0,0.000000,0.000000,0.991741,0.00000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.0,0.988309,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.962493,0.000000,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.988155,0.000000,0.0,0.000000
4,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.981995,0.0,0.000000
5,0.0,0.000000,0.544024,0.000000,0.00000,0.000000,0.440588,0.000000,0.0,0.000000
6,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.955859,0.000000,0.0,0.000000
7,0.0,0.000000,0.000000,0.000000,0.99126,0.000000,0.000000,0.000000,0.0,0.000000
8,0.0,0.000000,0.443141,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.545589
9,0.0,0.000000,0.000000,0.000000,0.00000,0.989652,0.000000,0.000000,0.0,0.000000


In [ ]:
ya_topics = (ya_df[['gr_id', 'title', 'publisher', 'clean_text', 'avg_rating']]
                  .reset_index(drop=False).merge(threshold_topics.reset_index(drop=False), on='index', how='inner'))
adventure = ya_topics[ya_topics.Adventure > 0]
love = ya_topics[ya_topics.Love > 0]
mysterious = ya_topics[ya_topics.Mysterious > 0]
life= ya_topics[ya_topics.Life > 0]
highschool = ya_topics[ya_topics.Highschool > 0]
transition = ya_topics[ya_topics.Transition > 0]
tragedy = ya_topics[ya_topics.Tragedy > 0]
friendship = ya_topics[ya_topics.Friendship > 0]
family = ya_topics[ya_topics.Family > 0]
drama = ya_topics[ya_topics.Drama > 0]
adventure.head()

,index,gr_id,title,publisher,clean_text,avg_rating,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
15,15,1656001,The Host,"Little, Brown and Company",melanie stryder refuses fade away earth invade...,3.84,0.983925,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,30,7736182,"The Lost Hero (The Heroes of Olympus, #1)",Disney ● Hyperion Books,jason problem doesnt remember anything waking ...,4.33,0.415279,0.579046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,39,2998,The Secret Garden,Children's Classics,one delightful enduring classics children lit...,4.13,0.706639,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,45,307791,"The City of Ember (Book of Ember, #1)",Random House Children's Books,an alternate cover edition found here many hun...,3.86,0.985480,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,55,50,"Hatchet (Brian's Saga, #1)",Atheneum Books for Young Readers: Richard Jack...,brian way canada visit estranged father pilot ...,3.73,0.981628,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
adventure['Topic'] = 'adventure'
love['Topic'] = 'love'
mysterious['Topic'] = 'mysterious'
life['Topic'] = 'life'
highschool['Topic'] = 'highschool'
transition['Topic'] = 'transition'
tragedy['Topic'] = 'tragedy'
friendship['Topic'] = 'friendship'
family['Topic'] = 'family'
drama['Topic'] = 'drama'
all_ya_topics = pd.concat([adventure,love,mysterious, life,highschool, transition, tragedy, friendship, family, drama])
# test4 = test3.groupby(['publisher', 'Topic']).size().reset_index().rename(columns={0:'num_books'}).head(20) #save this file
# test4 = test3.groupby(['publisher', 'Topic'])['avg_rating'].size().reset_index().rename(columns={0:'num_books'}).head(20)
all_ya_topics.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,index,gr_id,title,publisher,clean_text,avg_rating,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama,Topic
15,15,1656001,The Host,"Little, Brown and Company",melanie stryder refuses fade away earth invade...,3.84,0.983925,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adventure
30,30,7736182,"The Lost Hero (The Heroes of Olympus, #1)",Disney ● Hyperion Books,jason problem doesnt remember anything waking ...,4.33,0.415279,0.579046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adventure
37,39,2998,The Secret Garden,Children's Classics,one delightful enduring classics children lit...,4.13,0.706639,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adventure
43,45,307791,"The City of Ember (Book of Ember, #1)",Random House Children's Books,an alternate cover edition found here many hun...,3.86,0.985480,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adventure
53,55,50,"Hatchet (Brian's Saga, #1)",Atheneum Books for Young Readers: Richard Jack...,brian way canada visit estranged father pilot ...,3.73,0.981628,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adventure


In [ ]:
ya_pub_topics = all_ya_topics.groupby(['publisher', 'Topic']).agg({
    'title': 'count',
    'Adventure': 'mean',
    'Love': 'mean',
    'Mysterious': 'mean',
    'Life': 'mean',
    'Highschool': 'mean',
    'Transition': 'mean',
    'Tragedy': 'mean',
    'Friendship': 'mean',
    'Family': 'mean',
    'Drama': 'mean'
}).rename(columns={'title': 'num_books'})
ya_pub_topics.tail(10).reset_index()

,publisher,Topic,num_books,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
0,iBooks,transition,1,0.0,0.00000,0.00000,0.000000,0.0,0.984207,0.000000,0.000000,0.0,0.0
1,iUniverse,life,2,0.0,0.00000,0.00000,0.985647,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,iUniverse,love,2,0.0,0.97562,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,iUniverse,transition,2,0.0,0.00000,0.00000,0.000000,0.0,0.976234,0.000000,0.000000,0.0,0.0
4,iUniverse.com,tragedy,1,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.653989,0.000000,0.0,0.0
5,jimmy patterson,tragedy,1,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.986360,0.000000,0.0,0.0
6,self,tragedy,1,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.671555,0.000000,0.0,0.0
7,self published,life,1,0.0,0.00000,0.00000,0.984996,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,self-published,friendship,1,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,0.989998,0.0,0.0
9,voicecollective.press,mysterious,1,0.0,0.00000,0.98043,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [ ]:
ya_pub_topics.reset_index(drop=False).to_csv('test_ya_pub_topics.csv', index=False)

## Sci-Fi Books

In [ ]:
num_topics = 10

count_vectorizer = CountVectorizer(stop_words='english')
lda_data = count_vectorizer.fit_transform(sci_fi_df.clean_text)
# Create and fit the LDA model
lda = LDA(n_components=num_topics, n_jobs=-1)
lda.fit(lda_data)
#lda.components_[0]

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, 25)

Topics found via LDA:

Topic #0:
life world new man men love earth young like human save series war city years way finds long evil people time past story journey adventure

Topic #1:
world life new man war magic human old earth time young year love story city power people powerful like years finds strange planet god book

Topic #2:
world life time new war earth years space planet way story man book alien people future power human like secret death science young family fiction

Topic #3:
new war book world life time like dark long ancient fantasy magic man series novel good story earth years edition king anne people young finds

Topic #4:
world story new time life love human power man dark card death powerful orson book magic scott years like earth long city war people novel

Topic #5:
story new fiction science world stories life award man love time short future years novel human year old earth war book collection hugo power death

Topic #6:
new world story life war ender dark ray short

In [ ]:
# 0- Adventure ,1 - Mythical, 2 - Alien, 3 - Medieval ,  4 -Magical , 5 - Love , 6 - Space ,  7 - Heroic, 8 - War , 9 - Historical, 

In [ ]:
#Output1 - A file showing which words load on which topics, 
words = count_vectorizer.get_feature_names()
topic0 = lda.components_[0]
topic1 = lda.components_[1]
topic2 = lda.components_[2]
topic3 = lda.components_[3]
topic4 = lda.components_[4]
topic5 = lda.components_[5]
topic6 = lda.components_[6]
topic7 = lda.components_[7]
topic8 = lda.components_[8]
topic9 = lda.components_[9]
d = {'words': words,
     'Adventure': topic0,
     'Mythical':topic1,
     'Alien': topic2,
     'Medieval': topic3,
     'Magical': topic4,
     'Love': topic5,
     'Space': topic6,
     'Heroic': topic7,
     'War': topic8,
     'Historical': topic9     }
output1 = pd.DataFrame(d)
output1.head(10)

,words,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical
0,aachan,0.100000,0.1,0.100000,0.100000,0.100027,0.100000,0.100000,0.1,1.099973,0.1
1,aachim,0.100000,0.1,2.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,0.1
2,aakesh,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,1.1
3,aarabassa,0.100000,0.1,0.100000,0.100000,2.099991,0.100000,0.100009,0.1,0.100000,0.1
4,aaron,0.100003,0.1,3.099980,0.100000,0.100014,3.100003,0.100000,0.1,0.100000,0.1
5,aarons,0.100000,0.1,0.100000,0.100000,0.100000,2.100000,0.100000,0.1,0.100000,0.1
6,aastra,0.100000,0.1,1.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,0.1
7,ab,0.100000,0.1,0.100000,0.100000,0.100000,0.100000,0.100000,0.1,0.100000,1.1
8,ababa,0.100000,0.1,0.100000,0.100000,0.100000,1.100000,0.100000,0.1,0.100000,0.1
9,abaddon,0.100000,0.1,0.100049,1.099951,0.100000,0.100000,0.100000,0.1,0.100000,0.1


In [ ]:
weights=lda.transform(lda_data)
topic_weights=pd.DataFrame(weights).rename(columns={0:'Adventure', 1: 'Mythical', 2: 'Alien', 3:'Medieval', 4: 'Magical', 5:'Love', 6:'Space', 7: 'Heroic', 8: 'War', 9:'Historical'})
topic_weights.head(10)

,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical
0,0.000807,0.000807,0.000807,0.000807,0.000807,0.000807,0.992740,0.000807,0.000807,0.000807
1,0.001755,0.001755,0.713711,0.001755,0.001755,0.272251,0.001755,0.001755,0.001755,0.001755
2,0.001923,0.001923,0.982688,0.001923,0.001924,0.001924,0.001924,0.001923,0.001924,0.001924
3,0.002128,0.002128,0.980847,0.002128,0.002128,0.002128,0.002128,0.002128,0.002129,0.002128
4,0.001450,0.001450,0.001450,0.001449,0.001450,0.001450,0.001450,0.986954,0.001450,0.001450
5,0.002942,0.002942,0.474471,0.002942,0.386033,0.118904,0.002942,0.002942,0.002942,0.002942
6,0.001316,0.001316,0.001316,0.001316,0.001316,0.988155,0.001316,0.001316,0.001316,0.001316
7,0.003334,0.003334,0.003334,0.003334,0.003334,0.003334,0.003334,0.003335,0.969993,0.003334
8,0.002000,0.002000,0.002001,0.002000,0.981996,0.002000,0.002000,0.002000,0.002000,0.002000
9,0.000763,0.000763,0.000764,0.000763,0.000764,0.000764,0.000763,0.993128,0.000764,0.000763


In [ ]:
threshold_topics = topic_weights.apply(lambda x: np.where(x >= 0.4, x, 0))
threshold_topics.head(10)

,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical
0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.99274,0.000000,0.000000,0.0
1,0.0,0.0,0.713711,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0
2,0.0,0.0,0.982688,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0
3,0.0,0.0,0.980847,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.986954,0.000000,0.0
5,0.0,0.0,0.474471,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0
6,0.0,0.0,0.000000,0.0,0.000000,0.988155,0.00000,0.000000,0.000000,0.0
7,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.969993,0.0
8,0.0,0.0,0.000000,0.0,0.981996,0.000000,0.00000,0.000000,0.000000,0.0
9,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.993128,0.000000,0.0


In [ ]:
scifi_topics = (sci_fi_df[['gr_id', 'title', 'publisher', 'clean_text', 'avg_rating']]
                  .reset_index(drop=False).merge(threshold_topics.reset_index(drop=False), on='index', how='inner'))
adventure = scifi_topics[scifi_topics.Adventure > 0]
mythical = scifi_topics[scifi_topics.Mythical > 0]
alien = scifi_topics[scifi_topics.Alien > 0]
medieval = scifi_topics[scifi_topics.Medieval > 0]
magical = scifi_topics[scifi_topics.Magical > 0]
love = scifi_topics[scifi_topics.Love > 0]
space = scifi_topics[scifi_topics.Space > 0]
heroic = scifi_topics[scifi_topics.Heroic > 0]
war = scifi_topics[scifi_topics.War > 0]
historical = scifi_topics[scifi_topics.Historical > 0]
alien.head()

,index,gr_id,title,publisher,clean_text,avg_rating,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical
1,1,44767458,"Dune (Dune, #1)",Ace Books,set desert planet arrakis dune story boy paul ...,4.23,0.0,0.0,0.713711,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,40961427,1984,Houghton Mifflin Harcourt,among seminal texts century nineteen eighty fo...,4.19,0.0,0.0,0.982688,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,29579,"Foundation (Foundation, #1)",Bantam,twelve thousand years galactic empire ruled su...,4.17,0.0,0.0,0.474471,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Del Rey Books,last paperback one complete volume five novels...,4.36,0.0,0.0,0.957136,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,13,41804,"I, Robot (Robot, #0.1)",Bantam Books,isaac asimov i robot launches readers adventur...,4.20,0.0,0.0,0.991427,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
adventure['Topic'] = 'adventure'
mythical['Topic'] = 'mythical'
alien['Topic'] = 'alien'
medieval['Topic'] = 'medieval'
magical['Topic'] = 'magical'
love['Topic'] = 'love'
space['Topic'] = 'space'
heroic['Topic'] = 'heroic'
war['Topic'] = 'war'
historical['Topic'] = 'historical'
all_scifi_topics = pd.concat([adventure, mythical, alien, medieval, magical, love, space, heroic, war, historical])
all_scifi_topics.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,index,gr_id,title,publisher,clean_text,avg_rating,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical,Topic
16,17,7967,"Speaker for the Dead (Ender's Saga, #2)",Tor Books,available mass market revised definitive editi...,4.07,0.979995,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,adventure
18,19,76778,The Martian Chronicles,Bantam/Spectra,strange wonderful tale mans experiences mars f...,4.14,0.520029,0.0,0.000000,0.0,0.0,0.46727,0.0,0.0,0.0,0.0,adventure
26,28,18373,Flowers for Algernon,Harvest Books,story mentally disabled man whose experimental...,4.14,0.411529,0.0,0.573082,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,adventure
41,44,51964,"Old Man's War (Old Man's War, #1)",Tor Books,john perry two things birthday first visited w...,4.22,0.671163,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,adventure
51,55,334176,"The Sparrow (The Sparrow, #1)",Ballantine Books,humanity finally finds proof extraterrestrial...,4.12,0.981246,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,adventure


In [ ]:
scifi_pub_topics = all_scifi_topics.groupby(['publisher', 'Topic']).agg({
    'title': 'count',
    'Adventure': 'mean',
    'Mythical': 'mean',
    'Alien': 'mean',
    'Medieval': 'mean',
    'Magical': 'mean',
    'Love': 'mean',
    'Space': 'mean',
    'Heroic': 'mean',
    'War': 'mean',
    'Historical': 'mean'
}).rename(columns={'title': 'num_books'})
scifi_pub_topics.tail(10).reset_index()

,publisher,Topic,num_books,Adventure,Mythical,Alien,Medieval,Magical,Love,Space,Heroic,War,Historical
0,hpmor.com & fanfiction.net,mythical,1,0.000000,0.987321,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,iBooks,adventure,2,0.789337,0.000000,0.000000,0.000000,0.0,0.200587,0.0,0.000000,0.000000,0.0
2,iBooks,alien,3,0.000000,0.000000,0.929588,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,iBooks,heroic,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.980431,0.000000,0.0
4,iBooks,love,2,0.294760,0.000000,0.000000,0.000000,0.0,0.694742,0.0,0.000000,0.000000,0.0
5,iBooks,mythical,1,0.000000,0.989653,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
6,iBooks,war,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.989533,0.0
7,ibooks Inc.,medieval,1,0.000000,0.000000,0.000000,0.650754,0.0,0.000000,0.0,0.000000,0.000000,0.0
8,jimmy patterson,heroic,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.825601,0.000000,0.0
9,self,heroic,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.978044,0.000000,0.0


In [ ]:
scifi_pub_topics.reset_index(drop=False).to_csv('test_scifi_pub_topics.csv', index=False)

In [ ]:
# #get all topics with score greater than a threshold(0.4)
# book_topics = topic_weights.gt(0.4, 0)
# book_topics = pd.DataFrame(book_topics.apply(lambda x: ' '.join(x.index[x]),axis=1))
# topic_weights['Topics'] = book_topics
# topic_weights.loc[35:42]

In [ ]:
# test_gr = sci_fi_df.reset_index(drop=False)
# test_topics = topic_weights.reset_index(drop=False)
# book_weights = test_gr.merge(test_topics, on='index', how='inner')
# book_weights.head()

In [ ]:
# #aggregate by publisher
# publisher_topics = book_weights[['publisher','Topics']]
# publisher_topics = publisher_topics.groupby('publisher').agg(lambda x: " ".join(set(x)))
# publisher_topics.head()

In [ ]:
# publisher_topics_count = publisher_topics.groupby(['publisher','Topics']).size().reset_index()
# len(publisher_topics_count)

In [ ]:
# publisher_topics1 = publisher_topics.reset_index(drop=False)
# publisher_topics1.head()

In [ ]:
# #removing the duplicate topic names from the 'Topics' column
# # test with row 760 for duplicates
# from collections import defaultdict 
# d = defaultdict(list)
# top = []
# uniqtopics = []
# Topicnames = ['Aliens', 'Historical', 'War', 'Heroic', 'Medieval', 'Evil', 'Love', 'Space','Magical', 'Fantasy']
# for i in range(0, len(publisher_topics)):
#   x = str(publisher_topics1.iloc[i,1])
#   pubname = publisher_topics1.iloc[i,0]
#   #print(x)
#   #if "," in x :
#   top = x.split(' ')
#   #print(top)
#   for item in top:
#     if item not in uniqtopics:# and item in Topicnames :
#       uniqtopics.append(item)
#       #print(uniqtopics)
#       d[pubname] = uniqtopics
#   #publisher_topics.loc[i,'Topics'] = topics #Not sure how to assign the cleaned topic list back to df

# #publisher_topics.head()

In [ ]:
# pubtopics = pd.DataFrame()
# for key,value in d.items():
#   test = [key] * len(value)
#   tempdf = pd.DataFrame({'publisher':test, 'topic': value}) 
#   pubtopics = pd.concat([pubtopics,tempdf])
# pubtopics = pubtopics.reset_index(drop= True)
# pubtopics.head(20)  

In [ ]:
# pubtopics.groupby('topic').count()

In [ ]:
# pubtopics.publisher.unique()

In [ ]:
# publisher_topics.to_csv("publisher_topics_clean.csv")

## Mystery Books

In [ ]:
num_topics = 5

count_vectorizer = CountVectorizer(stop_words='english')
lda_data = count_vectorizer.fit_transform(mystery_df.clean_text)
# Create and fit the LDA model
lda = LDA(n_components=num_topics, n_jobs=-1)
lda.fit(lda_data)
#lda.components_[0]

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, 25)

Topics found via LDA:

Topic #0:
life world man new novel woman death love dark old murder crime like dead day young town story soon city finds best war way family

Topic #1:
new life book murder man time story crime years novel world finds love series woman young death author secret old york soon way detective people

Topic #2:
new man world time life murder mystery family detective novel old killer case author dead city crime secret like death story way young finds book

Topic #3:
world life man dead new series people city time sam old death young love murder book detective like jack novel father years long case dangerous

Topic #4:
life new murder killer man years family young case police old detective time world past story death dead crime woman novel year home love girl


In [ ]:
# 0 - Crime , 1 - Love, 2-Detective,  3 - Adventure, 4- Family

In [ ]:
#Output1 - A file showing which words load on which topics, 
words = count_vectorizer.get_feature_names()
topic0 = lda.components_[0]
topic1 = lda.components_[1]
topic2 = lda.components_[2]
topic3 = lda.components_[3]
topic4 = lda.components_[4]
d = {'words': words,
     'Crime': topic0,
     'Love':topic1,
     'Detective': topic2,
     'Adventure': topic3,
     'Family': topic4
      }
output1 = pd.DataFrame(d)
output1.head(10)

,words,Crime,Love,Detective,Adventure,Family
0,aa,0.200000,0.200000,0.200000,1.198307,0.201692
1,aalk,0.200000,0.200000,0.200000,1.199999,0.200000
2,aaron,0.200001,0.200001,0.200001,0.200962,6.199035
3,ababa,1.199999,0.200000,0.200000,0.200000,0.200000
4,abandon,1.626755,2.200364,1.985737,0.200228,3.986915
5,abandoned,11.277404,14.787587,6.706787,6.929430,39.298792
6,abandonment,2.069164,0.200001,0.200001,0.200002,1.330831
7,abandons,1.200052,1.199947,0.200000,0.200001,0.200000
8,abarca,0.200000,0.200000,0.200000,0.200001,1.199998
9,abbassati,0.200001,0.200001,0.200001,0.200001,1.199996


In [ ]:
weights=lda.transform(lda_data)
topic_weights=pd.DataFrame(weights).rename(columns={0:'Crime', 1: 'Love', 2: 'Detective', 3:'Adventure', 4: 'Family'})
topic_weights.head(10)

,Crime,Love,Detective,Adventure,Family
0,0.003098,0.003102,0.292790,0.003104,0.697906
1,0.001762,0.001762,0.992947,0.001760,0.001770
2,0.003231,0.003226,0.987087,0.003219,0.003237
3,0.004666,0.004688,0.004649,0.004640,0.981357
4,0.987405,0.003135,0.003163,0.003114,0.003184
5,0.003742,0.003707,0.985140,0.003680,0.003731
6,0.004255,0.004269,0.004265,0.004209,0.983002
7,0.003695,0.985147,0.003747,0.003697,0.003714
8,0.001181,0.872124,0.001184,0.001178,0.124333
9,0.003179,0.003199,0.003194,0.003159,0.987268


In [ ]:
threshold_topics = topic_weights.apply(lambda x: np.where(x >= 0.4, x, 0))
threshold_topics.head(10)

,Crime,Love,Detective,Adventure,Family
0,0.000000,0.000000,0.000000,0.0,0.697906
1,0.000000,0.000000,0.992947,0.0,0.000000
2,0.000000,0.000000,0.987087,0.0,0.000000
3,0.000000,0.000000,0.000000,0.0,0.981357
4,0.987405,0.000000,0.000000,0.0,0.000000
5,0.000000,0.000000,0.985140,0.0,0.000000
6,0.000000,0.000000,0.000000,0.0,0.983002
7,0.000000,0.985147,0.000000,0.0,0.000000
8,0.000000,0.872124,0.000000,0.0,0.000000
9,0.000000,0.000000,0.000000,0.0,0.987268


In [ ]:
mystery_topics = (mystery_df[['gr_id', 'title', 'publisher', 'clean_text', 'avg_rating']]
                  .reset_index(drop=False).merge(threshold_topics.reset_index(drop=False), on='index', how='inner'))
crime = mystery_topics[mystery_topics.Crime > 0]
love = mystery_topics[mystery_topics.Love > 0]
detective = mystery_topics[mystery_topics.Detective > 0]
adventure = mystery_topics[mystery_topics.Adventure > 0]
family = mystery_topics[mystery_topics.Family > 0]
crime.head()

,index,gr_id,title,publisher,clean_text,avg_rating,Crime,Love,Detective,Adventure,Family
4,4,168642,In Cold Blood,Vintage,november small town holcomb kansas four member...,4.07,0.987405,0.0,0.0,0.0,0.000000
11,11,1232,The Shadow of the Wind (The Cemetery of Forgot...,Penguin Books,barcelona city slowly heals war wounds daniel ...,4.27,0.549511,0.0,0.0,0.0,0.439931
13,13,2052,The Big Sleep,Vintage Crime,down mean streets man must go mean neither tar...,3.99,0.985642,0.0,0.0,0.0,0.000000
17,17,28877,"Red Dragon (Hannibal Lecter, #1)",Dutton,second family massacred terrifying serial kill...,4.03,0.782787,0.0,0.0,0.0,0.000000
19,20,40440,The Thirteenth Tale,Atria Books,children mythologize birth so begins prologue ...,3.96,0.859511,0.0,0.0,0.0,0.000000


In [ ]:
crime['Topic'] = 'crime'
love['Topic'] = 'love'
detective['Topic'] = 'detective'
adventure['Topic'] = 'adventure'
family['Topic'] = 'family'

all_mystery_topics = pd.concat([crime, love, detective, adventure, family])
# test4 = test3.groupby(['publisher', 'Topic']).size().reset_index().rename(columns={0:'num_books'}).head(20) #save this file
# test4 = test3.groupby(['publisher', 'Topic'])['avg_rating'].size().reset_index().rename(columns={0:'num_books'}).head(20)
all_mystery_topics.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,index,gr_id,title,publisher,clean_text,avg_rating,Crime,Love,Detective,Adventure,Family,Topic
4,4,168642,In Cold Blood,Vintage,november small town holcomb kansas four member...,4.07,0.987405,0.0,0.0,0.0,0.000000,crime
11,11,1232,The Shadow of the Wind (The Cemetery of Forgot...,Penguin Books,barcelona city slowly heals war wounds daniel ...,4.27,0.549511,0.0,0.0,0.0,0.439931,crime
13,13,2052,The Big Sleep,Vintage Crime,down mean streets man must go mean neither tar...,3.99,0.985642,0.0,0.0,0.0,0.000000,crime
17,17,28877,"Red Dragon (Hannibal Lecter, #1)",Dutton,second family massacred terrifying serial kill...,4.03,0.782787,0.0,0.0,0.0,0.000000,crime
19,20,40440,The Thirteenth Tale,Atria Books,children mythologize birth so begins prologue ...,3.96,0.859511,0.0,0.0,0.0,0.000000,crime


In [ ]:
mystery_pub_topics = all_mystery_topics.groupby(['publisher', 'Topic']).agg({
    'title': 'count',
    'Crime': 'mean',
    'Love': 'mean',
    'Detective': 'mean',
    'Adventure': 'mean',
    'Family': 'mean'
}).rename(columns={'title': 'num_books'})
mystery_pub_topics.tail(10).reset_index()

,publisher,Topic,num_books,Crime,Love,Detective,Adventure,Family
0,createspace,family,1,0.0,0.0,0.000000,0.0,0.983017
1,iBooks,family,2,0.0,0.0,0.000000,0.0,0.989575
2,iUniverse,detective,1,0.0,0.0,0.990019,0.0,0.000000
3,iUniverse,family,1,0.0,0.0,0.000000,0.0,0.741479
4,kdp,family,1,0.0,0.0,0.000000,0.0,0.987652
5,lulu.com,detective,1,0.0,0.0,0.483674,0.0,0.000000
6,lulu.com,family,1,0.0,0.0,0.000000,0.0,0.993792
7,self published E Book,family,1,0.0,0.0,0.000000,0.0,0.828538
8,stargazer press,detective,1,0.0,0.0,0.506737,0.0,0.488945
9,stargazer press,family,1,0.0,0.0,0.506737,0.0,0.488945


In [ ]:
mystery_pub_topics.reset_index(drop=False).to_csv('test_mystery_pub_topics.csv', index=False)

# Network Analysis

In [ ]:
def make_matrix(pubtop, pub_count):

  #Nadia's version of Palmer'code -- matrix values are number of shared topics
  start_time = time.time()
  pub_sim_matrix = pd.DataFrame(pub_count['publisher'])
  for i, row in pub_count.iterrows():
      pub_sim_matrix[row[0]] = 0
  for i, row in pub_sim_matrix.iterrows():
      top_count1 = []
      for item in range (0, len(pubtop[row[0]])):
          top_count1.append(list(pubtop[row[0]][item].values())[1])
      for column in pub_sim_matrix:
          if column == 'publisher':
              continue
          top_count2 = []
          for item in range (0, len(pubtop[column])):
              top_count2.append(list(pubtop[column][item].values())[1])
          sim_count = 0
          for topic in top_count1:
              if topic in top_count2:
                  sim_count = sim_count + 1
          # sim_score = sim_count / (len(top_count1) + len(top_count2) - sim_count)
          t = [topic for topic in top_count1 if topic in top_count2]
          pub_sim_matrix.loc[i, column] = len(t)
      if i % 50 == 0:
          print("finished " + row[0] + " (row " + str(i) + "/" + str(len(pub_sim_matrix)-1) + ")")
          print("--- %s seconds ---" % (time.time() - start_time))
  return pub_sim_matrix

def make_edge_list(pub_sim_matrix, threshold):

  ## if number of topics in common > threshold then create an edge
  for_network_df = pub_sim_matrix.set_index('publisher')
  cols = for_network_df.columns

  edge_list = [] #holds edges
  for index, row in for_network_df.iterrows():
    for publisher in cols:
      val = row[publisher]
      if index != publisher and val > threshold:
        edge_list.append((index, publisher, {'weight': val}))
  print(edge_list[0:15])

  return edge_list

def make_graph(pub_sim_matrix, threshold):
  edge_list = make_edge_list(pub_sim_matrix, threshold)
  G = nx.Graph()
  G.add_nodes_from(pub_sim_matrix.columns)
  G.add_edges_from(edge_list)
  return G

In [ ]:
def topics_per_publisher(df):
  ## Palmer's version
  pubtop = df[['publisher','Topic']].groupby('publisher').apply(lambda x: x.to_dict('r')).to_dict()
  pubCount = df.groupby('publisher').agg({'Topic':'count'}).reset_index()
  pub_count = pubCount[pubCount.Topic > 1].reset_index(drop=True)
  return pubtop, pub_count


### Young Adult

In [ ]:
ya = pd.read_csv('test_ya_pub_topics1.csv')
scifi = pd.read_csv('test_scifi_pub_topics1.csv')
mys = pd.read_csv('test_mystery_pub_topics1.csv')

In [ ]:
ya

,publisher,Topic,num_books,Adventure,Love,Mysterious,Life,Highschool,Transition,Tragedy,Friendship,Family,Drama
0,18th Avenue Press,mysterious,1,0.0,0.0,0.641667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
1,215 Ink,life,1,0.0,0.0,0.000000,0.978044,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
2,215 Ink,mysterious,1,0.0,0.0,0.989770,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
3,47North,family,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.98043,0.0
4,48fourteen,highschool,1,0.0,0.0,0.000000,0.000000,0.544696,0.445301,0.000000,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,jimmy patterson,tragedy,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.986360,0.000000,0.00000,0.0
1885,self,tragedy,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.671555,0.000000,0.00000,0.0
1886,self published,life,1,0.0,0.0,0.000000,0.984996,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
1887,self-published,friendship,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.989998,0.00000,0.0


In [ ]:
len(ya.publisher.unique())

1889

In [ ]:
 #note: also reduces dataframe to those publishers with more than 1 topic
ya_pubtop, ya_pub_count = topics_per_publisher(ya)
scifi_pubtop, scifi_pub_count = topics_per_publisher(scifi)
mys_pubtop, mys_pub_count = topics_per_publisher(mys)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [ ]:
ya_pub_count[ya_pub_count.Topic > 2]

,publisher,Topic
2,Ace,7
3,Ace Books,7
4,Aladdin,9
5,Aladdin Paperbacks,4
6,Alfred A. Knopf,6
...,...,...
312,William Morrow Paperbacks,3
313,Wizards of the Coast,4
316,Yearling,6
317,Yearling Books,3


In [ ]:
scifi_pub_count[scifi_pub_count.Topic > 2]

,publisher,Topic
1,Abbott Press,3
2,Ace,10
3,Ace Books,10
5,Ace Hardcover,3
6,Aladdin,3
...,...,...
259,Wildside Press,4
260,William Morrow,4
261,William Morrow Paperbacks,5
262,Wizards of the Coast,6


In [ ]:
mys_pub_count[mys_pub_count.Topic > 2] #threshold for mystery

,publisher,Topic
3,Alfred A. Knopf,3
4,Algonquin Books,3
6,Amazon Digital Services,5
7,Anchor,3
8,Anchor Books,3
...,...,...
229,W. W. Norton Company,5
233,Warner Books (NY),3
237,William Morrow,4
240,Zebra,3


In [ ]:
# find connected publishers
ya[ya.publisher.isin(ya_publishers)].groupby('Topic').agg({'publisher': 'count'}) #old code need to update

,publisher
Topic,
adventure,182
ancestry,139
classic,71
family,141
friendship,165
highschool,96
love,90
medieval,149
tragedy,86


In [ ]:
## create matrix
ya_pub_sim_matrix = make_matrix(ya_pubtop, ya_pub_count)
scifi_pub_sim_matrix = make_matrix(scifi_pubtop, scifi_pub_count)
mys_pub_sim_matrix = make_matrix(mys_pubtop, mys_pub_count)

finished 215 Ink (row 0/320)
--- 0.29008007049560547 seconds ---
finished Create Space (row 50/320)
--- 4.262511730194092 seconds ---
finished Farrar, Straus and Giroux (BYR) (row 100/320)
--- 8.252225160598755 seconds ---
finished Harvest Books (row 150/320)
--- 12.255898475646973 seconds ---
finished Norton (row 200/320)
--- 16.17532181739807 seconds ---
finished Shadow Mountain (row 250/320)
--- 20.021297216415405 seconds ---
finished W. W. Norton & Company (row 300/320)
--- 23.90132236480713 seconds ---
finished 21st Century Lion Books (row 0/266)
--- 0.19939208030700684 seconds ---
finished Bronwyn Editions (row 50/266)
--- 3.5008463859558105 seconds ---
finished Friesenpress (row 100/266)
--- 6.669648170471191 seconds ---
finished Little, Brown and Company (row 150/266)
--- 9.95120096206665 seconds ---
finished Red Fox (row 200/266)
--- 13.235944032669067 seconds ---
finished Viking Adult (row 250/266)
--- 16.47149419784546 seconds ---
finished Ace Books (row 0/236)
--- 0.1725864

In [ ]:
#Decomposed bi-partite network to publish-to-publisher network connected by # of topics
G_ya = make_graph(ya_pub_sim_matrix, threshold=2) #topics
G_scifi = make_graph(scifi_pub_sim_matrix, threshold=2) #10 topics
G_mys = make_graph(mys_pub_sim_matrix, threshold=2) #5 topics

[('Ace', 'Ace Books', {'weight': 6}), ('Ace', 'Aladdin', {'weight': 6}), ('Ace', 'Aladdin Paperbacks', {'weight': 4}), ('Ace', 'Alfred A. Knopf', {'weight': 5}), ('Ace', 'Alfred A. Knopf Books for Young Readers', {'weight': 4}), ('Ace', 'Algonquin Young Readers', {'weight': 3}), ('Ace', 'Amazon', {'weight': 4}), ('Ace', 'Amazon Digital Services', {'weight': 5}), ('Ace', 'Arthur A. Levine Books', {'weight': 6}), ('Ace', 'Atheneum Books for Young Readers', {'weight': 6}), ('Ace', 'Atria Books', {'weight': 6}), ('Ace', 'BLVNP Incorporated', {'weight': 3}), ('Ace', 'Ballantine Books', {'weight': 6}), ('Ace', 'Balzer + Bray', {'weight': 6}), ('Ace', 'Bantam', {'weight': 7})]
[('Abbott Press', 'Ace', {'weight': 3}), ('Abbott Press', 'Ace Books', {'weight': 3}), ('Abbott Press', 'Amazon Digital Services', {'weight': 3}), ('Abbott Press', 'Avon Books', {'weight': 3}), ('Abbott Press', 'Baen', {'weight': 3}), ('Abbott Press', 'Bantam Books', {'weight': 3}), ('Abbott Press', 'Bantam Spectra', {'

In [ ]:
print("YA nodes: {}".format(G_ya.number_of_nodes()))
print("YA edges: {}".format(G_ya.number_of_edges()))

YA nodes: 322
YA edges: 21887


In [ ]:
print("Scifi nodes: {}".format(G_scifi.number_of_nodes()))
print("Scifi edges: {}".format(G_scifi.number_of_edges()))

Scifi nodes: 268
Scifi edges: 2262


In [ ]:
print("Mys nodes: {}".format(G_mys.number_of_nodes()))
print("Mys edges: {}".format(G_mys.number_of_edges()))

Mys nodes: 238
Mys edges: 4824


In [ ]:
##YA
comp = girvan_newman(G_ya)
communities = tuple(sorted(c) for c in next(comp))
main_communities = [c for c in communities if len(c) > 1] # one community with 95 / 321 publishers
len(main_communities[0]) # what are these 93 publishers publishing on???? what makes them a community?

188

In [ ]:
##SciFI
comp = girvan_newman(G_scifi)
communities = tuple(sorted(c) for c in next(comp))
main_communities = [c for c in communities if len(c) > 1] # one community with 91 / 92 publishers
len(main_communities[0])

122

In [ ]:
##Mys
comp = girvan_newman(G_mys)
communities = tuple(sorted(c) for c in next(comp))
main_communities = [c for c in communities if len(c) > 1] # one community with 120 / 121 publishers
len(main_communities[0]) 

112

In [ ]:
len(main_communities)

1

In [ ]:
#https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html

In [ ]:
def pub_preproc(x):
  x = x.lower()
  x = re.sub(r' ', '', x)
  x = re.sub(r'-', '', x)
  return x

df['clean_publisher'] = df.publisher.apply(pub_preproc)

# Visualization

In [ ]:
#creates a list 
import csv
publist = ["publisher", "topic"]
for i in ya_pubtop.values():
    for j in range (0,len(i)):
        publist.append(list(list(i)[j].values()))
file = open('publist.csv', 'w+', newline ='', encoding = 'utf-8')
# writing the data into the file
with file:
    write = csv.writer(file)
    write.writerows(publist)

### Old code

In [ ]:
#Palmer
import time
start_time = time.time()
pub_sim_matrix = pd.DataFrame(pub_count['publisher'])
for i, row in pub_count.iterrows():
    pub_sim_matrix[row[0]] = 0
for i, row in pub_sim_matrix.iterrows():
    top_count1 = []
    for item in range(0, len(pubtop[row[0]])):
        top_count1.append(list(pubtop[row[0]][item].values())[1])
    for column in pub_sim_matrix:
        if column == 'publisher':
            continue
        top_count2 = []
        for item in range (0, len(pubtop[column])):
            top_count2.append(list(pubtop[column][item].values())[1])
        sim_count = 0
        for topic in top_count1:
            if topic in top_count2:
                sim_count = sim_count + 1
        sim_score = sim_count / (len(top_count1) + len(top_count2) - sim_count)
        pub_sim_matrix.loc[i, column] = round(sim_score,2)
    if i % 10 == 0:
        print("finished " + row[0] + " (row " + str(i) + "/" + str(len(pub_sim_matrix)-1) + ")")
        print("--- %s seconds ---" % (time.time() - start_time))
#print(pub_sim_matrix.loc[row_index, row[0]])


pub_sim_matrix.head() 

In [ ]:
#Nadia's version of Palmer'code -- matrix values are number of shared topics
start_time = time.time()
pub_sim_matrix = pd.DataFrame(pub_count['publisher'])
for i, row in pub_count.iterrows():
    pub_sim_matrix[row[0]] = 0
for i, row in pub_sim_matrix.iterrows():
    top_count1 = []
    for item in range (0, len(pubtop[row[0]])):
        top_count1.append(list(pubtop[row[0]][item].values())[1])
    for column in pub_sim_matrix:
        if column == 'publisher':
            continue
        top_count2 = []
        for item in range (0, len(pubtop[column])):
            top_count2.append(list(pubtop[column][item].values())[1])
        sim_count = 0
        for topic in top_count1:
            if topic in top_count2:
                sim_count = sim_count + 1
        # sim_score = sim_count / (len(top_count1) + len(top_count2) - sim_count)
        t = [topic for topic in top_count1 if topic in top_count2]
        pub_sim_matrix.loc[i, column] = len(t)
    if i % 10 == 0:
        print("finished " + row[0] + " (row " + str(i) + "/" + str(len(pub_sim_matrix)-1) + ")")
        print("--- %s seconds ---" % (time.time() - start_time))

pub_sim_matrix.head() 

In [ ]:
## if number of topics in common > 6 then create an edge
for_network_df = pub_sim_matrix.set_index('publisher')
cols = for_network_df.columns

edge_list = [] #holds edges
for index, row in for_network_df.iterrows():
  for publisher in cols:
    val = row[publisher]
    if index != publisher and val > 4:
      edge_list.append((index, publisher, {'weight': val}))
edge_list[0:15]